<a href="https://colab.research.google.com/github/nikhilmuser/AIML-project/blob/main/custom_model_foodimgrecog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.2 MB/s eta 0:00:00


In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!curl -s ipv4.icanhazip.com

34.30.194.89


In [ ]:
import numpy as np
import streamlit as st
import tensorflow as tf
from PIL import Image
import os

In [ ]:
import cv2
import os
import random
import collections
from collections import defaultdict

from shutil import copy
from shutil import copytree, rmtree

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as img

In [ ]:
print(tf.__version__)
print(tf.test.gpu_device_name())

2.15.0



In [ ]:
def get_data_extract():
  if "food-101" in os.listdir():
    print("Dataset already exists")
  else:
    tf.keras.utils.get_file(
    'food-101.tar.gz',
    'http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz',
    cache_subdir='/content',
    extract=True,
    archive_format='tar',
    cache_dir=None
    )
    print("Dataset downloaded and extracted!")

In [ ]:
get_data_extract()

4996278331/4996278331 [==============================] - 250s 0us/step
Dataset downloaded and extracted!


In [ ]:
def prepare_data(filepath, src,dest):
  classes_images = defaultdict(list)
  with open(filepath, 'r') as txt:
      paths = [read.strip() for read in txt.readlines()]
      for p in paths:
        food = p.split('/')
        classes_images[food[0]].append(food[1] + '.jpg')

  for food in classes_images.keys():
    print("\nCopying images into ",food)
    if not os.path.exists(os.path.join(dest,food)):
      os.makedirs(os.path.join(dest,food))
    for i in classes_images[food]:
      copy(os.path.join(src,food,i), os.path.join(dest,food,i))
  print("Copying Done!")

In [ ]:
print("Creating train data...")
prepare_data('food-101/meta/train.txt', 'food-101/images', 'food-101/train')

Creating train data...

Copying images into  apple_pie

Copying images into  baby_back_ribs

Copying images into  baklava

Copying images into  beef_carpaccio

Copying images into  beef_tartare

Copying images into  beet_salad

Copying images into  beignets

Copying images into  bibimbap

Copying images into  bread_pudding

Copying images into  breakfast_burrito

Copying images into  bruschetta

Copying images into  caesar_salad

Copying images into  cannoli

Copying images into  caprese_salad

Copying images into  carrot_cake

Copying images into  ceviche

Copying images into  cheesecake

Copying images into  cheese_plate

Copying images into  chicken_curry

Copying images into  chicken_quesadilla

Copying images into  chicken_wings

Copying images into  chocolate_cake

Copying images into  chocolate_mousse

Copying images into  churros

Copying images into  clam_chowder

Copying images into  club_sandwich

Copying images into  crab_cakes

Copying images into  creme_brulee

Copying im

In [ ]:
print("Creating test data...")
prepare_data('food-101/meta/test.txt', 'food-101/images', 'food-101/test')

Creating test data...

Copying images into  apple_pie

Copying images into  baby_back_ribs

Copying images into  baklava

Copying images into  beef_carpaccio

Copying images into  beef_tartare

Copying images into  beet_salad

Copying images into  beignets

Copying images into  bibimbap

Copying images into  bread_pudding

Copying images into  breakfast_burrito

Copying images into  bruschetta

Copying images into  caesar_salad

Copying images into  cannoli

Copying images into  caprese_salad

Copying images into  carrot_cake

Copying images into  ceviche

Copying images into  cheesecake

Copying images into  cheese_plate

Copying images into  chicken_curry

Copying images into  chicken_quesadilla

Copying images into  chicken_wings

Copying images into  chocolate_cake

Copying images into  chocolate_mousse

Copying images into  churros

Copying images into  clam_chowder

Copying images into  club_sandwich

Copying images into  crab_cakes

Copying images into  creme_brulee

Copying ima

In [ ]:
train_files = sum([len(files) for i, j, files in os.walk("food-101/train")])
print("Total number of samples in train folder")
print(train_files)

Total number of samples in train folder
75750


In [ ]:
test_files = sum([len(files) for i, j, files in os.walk("food-101/test")])
print("Total number of samples in test folder")
print(test_files)

Total number of samples in test folder
25250


In [ ]:
def dataset_mini(food_list, src, dest):
  if os.path.exists(dest):
    rmtree(dest) # removing dataset_mini(if it already exists) folders so that we will have only the classes that we want
  os.makedirs(dest)
  for food_item in food_list :
    print("Copying images into",food_item)
    copytree(os.path.join(src,food_item), os.path.join(dest,food_item))

In [ ]:
food_list = ['samosa','pizza','omelette']
src_train = 'food-101/train'
dest_train = 'food-101/train_mini'
src_test = 'food-101/test'
dest_test = 'food-101/test_mini'

In [ ]:
print("Creating train data folder with new classes")
dataset_mini(food_list, src_train, dest_train)

Creating train data folder with new classes
Copying images into samosa
Copying images into pizza
Copying images into omelette


In [ ]:
print("Total number of samples in train folder")
train_files = sum([len(files) for i, j, files in os.walk("food-101/train_mini")])
print(train_files)

Total number of samples in train folder
2250


In [ ]:
print("Creating test data folder with new classes")
dataset_mini(food_list, src_test, dest_test)

Creating test data folder with new classes
Copying images into samosa
Copying images into pizza
Copying images into omelette


In [ ]:
print("Total number of samples in test folder")
test_files = sum([len(files) for i, j, files in os.walk("food-101/test_mini")])
print(test_files)

Total number of samples in test folder
750


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
path_to_train_data = '/content/food-101/train_mini'  # Update this
path_to_valid_data = '/content/food-101/test_mini'  # Update this


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator

# Define the model
model = Sequential()
model.add(Conv2D(32, 3, padding="same", activation="relu", input_shape=(224, 224, 3)))
model.add(MaxPool2D())

model.add(Conv2D(32, 3, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Conv2D(64, 3, padding="same", activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dense(3, activation="softmax"))  # Assuming 3 classes in your dataset

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=10,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

train_generator = train_datagen.flow_from_directory(
    path_to_train_data,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)

valid_generator = test_datagen.flow_from_directory(
    path_to_valid_data,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)

# Train the custom model on your dataset
history=model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=valid_generator,
    validation_steps=len(valid_generator)
)

# Save the trained model
model.save('bestmodel.h5')


Found 2250 images belonging to 3 classes.
Found 750 images belonging to 3 classes.
Epoch 1/10
141/141 [==============================] - 220s 2s/step - loss: 1.1158 - accuracy: 0.3578 - val_loss: 1.0877 - val_accuracy: 0.3827
Epoch 2/10
141/141 [==============================] - 222s 2s/step - loss: 0.9857 - accuracy: 0.4818 - val_loss: 0.8748 - val_accuracy: 0.5800
Epoch 3/10
141/141 [==============================] - 220s 2s/step - loss: 0.9067 - accuracy: 0.5560 - val_loss: 0.7586 - val_accuracy: 0.6920
Epoch 4/10
141/141 [==============================] - 217s 2s/step - loss: 0.7866 - accuracy: 0.6422 - val_loss: 0.7223 - val_accuracy: 0.6733
Epoch 5/10
141/141 [==============================] - 228s 2s/step - loss: 0.7252 - accuracy: 0.6756 - val_loss: 0.7714 - val_accuracy: 0.6253
Epoch 6/10
141/141 [==============================] - 219s 2s/step - loss: 0.7110 - accuracy: 0.6907 - val_loss: 0.5435 - val_accuracy: 0.7907
Epoch 7/10
141/141 [==============================] - 230s 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [22]:
!pip install streamlit

In [23]:
!pip install gTTS

In [24]:
%%writefile aapp.py
import numpy as np
import streamlit as st
import tensorflow as tf
from PIL import Image, ImageOps
from gtts import gTTS

def import_and_predict(image_data, model):
    size = (299, 299)
    image = ImageOps.fit(image_data, size, method=Image.LANCZOS)
    image = image.convert('RGB')
    image = np.asarray(image)
    image = (image.astype(np.float32) / 255.0)
    img_reshape = image[np.newaxis, ...]
    prediction = model.predict(img_reshape)
    return prediction

model = tf.keras.models.load_model('/content/drive/MyDrive/bestmodel_6class.hdf5')


st.write("""
         # Food Prediction by Nikhil M
          Guide- Mr. Prasad K
         """
         )

st.write("This is a simple image classification web app to predict food item names")

file = st.file_uploader("Please upload an image file", type=["jpg", "png"])

food_classes = ['frenchfries', 'icecream', 'omelette', 'pizza', 'samosa', 'springrolls']

if file is None:
    st.text("You haven't uploaded an image file")
else:
    image = Image.open(file)
    st.image(image, use_column_width=True)
    prediction = import_and_predict(image, model)

    st.write("Prediction:")
    if np.argmax(prediction) == 0:
        st.write("It is french fries!")
    elif np.argmax(prediction) == 1:
        st.write("It is an ice cream!")
    elif np.argmax(prediction) == 2:
        st.write("It is an omelette!")
    elif np.argmax(prediction) == 3:
        st.write("It is a pizza!")
    elif np.argmax(prediction) == 4:
        st.write("It is a samosa!")
    elif np.argmax(prediction) == 5:
        st.write("It is spring rolls!")

    # Text-to-speech
    text_to_speech = f"It is a {food_classes[np.argmax(prediction)]}!"
    tts = gTTS(text=text_to_speech, lang='en')
    tts.save("output.mp3")

    # Play audio
    st.audio("output.mp3", format="audio/mp3", start_time=0)

    #st.write("Probability:")
    probabilities = {food_classes[i]: prediction[0][i] for i in range(len(food_classes))}
    #st.write(probabilities)


Overwriting aapp.py


In [7]:
!pip install streamlit

In [25]:
!curl -s ipv4.icanhazip.com

34.90.144.16


In [26]:
!streamlit run aapp.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.90.144.16:8501

npx: installed 22 in 4.991s
your url is: https://mean-sloths-lead.loca.lt
2024-05-29 17:25:56.583818: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-29 17:25:56.583882: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-29 17:25:56.585987: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-29 17:25:56.599429: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized

In [ ]:
import matplotlib.pyplot as plt

# Get the training history
training_loss = model.history.history['loss']
validation_loss = model.history.history['val_loss']
training_accuracy = model.history.history['accuracy']
validation_accuracy = model.history.history['val_accuracy']

# Number of epochs
epochs = range(1, len(training_loss) + 1)

# Plot training and validation loss
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs, training_loss, 'r', label='Training Loss')
plt.plot(epochs, validation_loss, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, training_accuracy, 'r', label='Training Accuracy')
plt.plot(epochs, validation_accuracy, 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


In [ ]:
from google.colab import files
files.download("/content/drive/MyDrive/bestmodel_6class.hdf5")